In [1]:
# === 步驟 1：安裝 nnU-Net V2 ===
# 安裝 PyTorch (這步通常你已經裝過了，確保一下)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 安裝 nnU-Net V2 及必要工具
!pip install nnunetv2
!pip install batchgenerators
!pip install medpy SimpleITK nibabel tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metada

In [1]:
# === 步驟 2：設定資料夾結構與環境變數 ===
import os

# 設定你的專案根目錄
base_dir = "C:/Users/user/AICUP"

# 定義 nnU-Net 需要的三個資料夾路徑
raw_data_dir = os.path.join(base_dir, "nnUNet_raw")
preprocessed_dir = os.path.join(base_dir, "nnUNet_preprocessed")
results_dir = os.path.join(base_dir, "nnUNet_results")

# 建立資料夾
os.makedirs(raw_data_dir, exist_ok=True)
os.makedirs(preprocessed_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# 設定環境變數 (讓 nnU-Net 知道資料放在哪)
os.environ["nnUNet_raw"] = raw_data_dir
os.environ["nnUNet_preprocessed"] = preprocessed_dir
os.environ["nnUNet_results"] = results_dir

print(f"nnU-Net 環境設定完成！根目錄位於: {os.path.abspath(base_dir)}")

nnU-Net 環境設定完成！根目錄位於: C:\Users\user\AICUP


In [9]:
# === 步驟 3：資料格式轉換 (修正 _gt 檔名對應版) ===
import shutil
import json
import os
import glob
from tqdm import tqdm

# 1. 設定來源資料路徑
source_data_dir = "C:/Users/user/AICUP/dataset"
train_img_src = os.path.join(source_data_dir, "Train")
train_lbl_src = os.path.join(source_data_dir, "Train_Labels")
test_img_src = os.path.join(source_data_dir, "Test")

# 2. 設定 nnU-Net 目標路徑
dataset_id = 501
dataset_name = "Cardiac"
nnunet_dataset_name = f"Dataset{dataset_id:03d}_{dataset_name}"
dataset_root = os.path.join(raw_data_dir, nnunet_dataset_name)

# 強制清理舊資料
if os.path.exists(dataset_root):
    print(f"清理舊資料夾: {dataset_root}")
    shutil.rmtree(dataset_root)

target_imagesTr = os.path.join(dataset_root, "imagesTr")
target_labelsTr = os.path.join(dataset_root, "labelsTr")
target_imagesTs = os.path.join(dataset_root, "imagesTs")

os.makedirs(target_imagesTr, exist_ok=True)
os.makedirs(target_labelsTr, exist_ok=True)
os.makedirs(target_imagesTs, exist_ok=True)

print(f"開始轉換資料至: {nnunet_dataset_name}")

# 3. 轉換訓練資料
train_files = sorted(glob.glob(os.path.join(train_img_src, "*.nii.gz")))

if len(train_files) == 0:
    raise FileNotFoundError(f"錯誤：在 {train_img_src} 找不到任何影像！")

missing_labels = []

for img_path in tqdm(train_files, desc="Converting Train Data"):
    filename = os.path.basename(img_path) # patient0001.nii.gz
    case_id = filename.replace(".nii.gz", "") # patient0001
    
    # A. 複製影像 -> nnU-Net 要求加上 _0000
    shutil.copy(img_path, os.path.join(target_imagesTr, f"{case_id}_0000.nii.gz"))
    
    # B. 複製標籤
    # 你的標籤檔名有 _gt，所以我們要手動加上去尋找
    expected_lbl_name = f"{case_id}_gt.nii.gz"
    lbl_path = os.path.join(train_lbl_src, expected_lbl_name)
    
    if os.path.exists(lbl_path):
        # 複製時要去掉 _gt，因為 nnU-Net 要求標籤檔名必須跟 Case ID 一模一樣
        shutil.copy(lbl_path, os.path.join(target_labelsTr, f"{case_id}.nii.gz"))
    else:
        missing_labels.append(expected_lbl_name)

# 檢查是否有標籤遺失
if len(missing_labels) > 0:
    print(f"\n錯誤！找不到以下 {len(missing_labels)} 個標籤檔案：")
    print(missing_labels[:5])
    raise FileNotFoundError("標籤缺失，請檢查檔名是否對應。")

# 4. 轉換測試資料
test_files = sorted(glob.glob(os.path.join(test_img_src, "*.nii.gz")))
for img_path in tqdm(test_files, desc="Converting Test Data"):
    filename = os.path.basename(img_path)
    case_id = filename.replace(".nii.gz", "")
    shutil.copy(img_path, os.path.join(target_imagesTs, f"{case_id}_0000.nii.gz"))

# 5. 生成 dataset.json
json_dict = {
    "channel_names": {"0": "CT"},
    "labels": {
        "background": 0,
        "muscle": 1,
        "valve": 2,
        "calcium": 3
    },
    "numTraining": len(train_files),
    "file_ending": ".nii.gz"
}

json_path = os.path.join(dataset_root, "dataset.json")
with open(json_path, 'w') as f:
    json.dump(json_dict, f, indent=4)

print("\n資料轉換成功！")

清理舊資料夾: C:/Users/user/AICUP\nnUNet_raw\Dataset501_Cardiac
開始轉換資料至: Dataset501_Cardiac


Converting Test Data: 100%|██████████| 50/50 [00:03<00:00, 12.88it/s]


資料轉換成功！已自動處理 _gt 後綴。
請重新執行 Step 4 (Plan & Preprocess)。


In [10]:
# === 步驟 4：執行 nnU-Net 自動規劃與預處理 ===
# -d 501: 指定 Dataset ID
# -c 3d_fullres: 指定使用 3D 全解析度模型 (4080 顯卡絕對跑得動，這是準確度最高的配置)
# --verify_dataset_integrity: 順便檢查資料有沒有壞掉

!nnUNetv2_plan_and_preprocess -d 501 -c 3d_fullres --verify_dataset_integrity

Fingerprint extraction...


100%|██████████| 50/50 [00:27<00:00,  1.81it/s]

100%|██████████| 50/50 [19:54<00:00, 23.89s/it]



Dataset501_Cardiac
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [0.515      0.40636719 0.40636719]. 
Current patch size: (np.int64(96), np.int64(160), np.int64(160)). 
Current median shape: [293.68932039 497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [0.53045   0.4185582 0.4185582]. 
Current patch size: (np.int64(96), np.int6

In [11]:
# === 步驟 5：全自動 5-Fold 訓練腳本 (含自定義 Log) ===
import os
import datetime
import subprocess
import sys
import time

# --- 1. 設定 Log 檔案名稱 ---
# 格式：日_時_分.txt (例如 22_16_30.txt)
current_time = datetime.datetime.now().strftime("%d_%H_%M")
log_filename = f"{current_time}.txt"
log_path = os.path.join(base_dir, log_filename) # 存放在專案根目錄

print(f"完整訓練紀錄將儲存於: {os.path.abspath(log_path)}")
print(f"準備開始 5-Fold Cross Validation (Fold 0 ~ 4) ...")
print("這將耗費非常長的時間，請確保電源連接穩定！")

# --- 2. 定義執行與紀錄函式 ---
def run_fold_and_log(fold, dataset_id, config):
    cmd = f"nnUNetv2_train {dataset_id} {config} {fold} --npz"
    
    header = f"\n{'='*40}\n   STARTING TRAINING FOR FOLD {fold}\n{'='*40}\n"
    print(header)
    
    # 開啟 Log 檔 (使用 'a' append 模式)
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(header)
        
        # 使用 subprocess 執行指令並即時抓取輸出
        process = subprocess.Popen(
            cmd, 
            shell=True, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, 
            text=True,
            encoding="utf-8",
            errors="replace"
        )
        
        # 逐行讀取輸出
        for line in process.stdout:
            # 1. 顯示在 Jupyter Terminal
            sys.stdout.write(line) 
            
            # 2. 寫入 Log 檔案
            f.write(line)
            
            # (選用) 強制刷新緩衝區，確保檔案即時更新
            if "Epoch" in line:
                f.flush()
        
        process.wait()
        
        if process.returncode != 0:
            error_msg = f"\nFold {fold} 訓練發生錯誤！Return Code: {process.returncode}\n"
            print(error_msg)
            f.write(error_msg)
            return False
        else:
            success_msg = f"\nFold {fold} 訓練完成！\n"
            print(success_msg)
            f.write(success_msg)
            return True

# --- 3. 開始依序訓練 Fold 0 ~ 4 ---
# 如果你想測試，可以先改成 [0] 跑跑看
folds_to_train = [0, 1, 2, 3, 4] 

dataset_id_code = 501  # 對應 Dataset501
config_mode = "3d_fullres"

start_time_total = time.time()

for fold in folds_to_train:
    success = run_fold_and_log(fold, dataset_id_code, config_mode)
    if not success:
        print(f"警告：Fold {fold} 失敗，腳本將終止。")
        break

end_time_total = time.time()
duration = end_time_total - start_time_total
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)

print(f"\n所有訓練任務結束！總耗時: {hours} 小時 {minutes} 分鐘")
print(f"請查看 Log 檔: {log_filename}")

完整訓練紀錄將儲存於: C:\Users\user\AICUP\22_17_47.txt
準備開始 5-Fold Cross Validation (Fold 0 ~ 4) ...
這將耗費非常長的時間，請確保電源連接穩定！

   STARTING TRAINING FOR FOLD 0


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-11-22 17:47:20.575509: do_dummy_2d_data_aug: False
2025-11-22 17:47:20.579511: Creating new 5-fold cross-validation split...
2025-11-22 17:47:20

In [2]:
# === 步驟 6：執行預測 ===
import os

# 定義輸入與輸出路徑
input_folder = os.path.join(raw_data_dir, "Dataset501_Cardiac", "imagesTs")
output_folder = os.path.join(results_dir, "Dataset501_Cardiac", "inference_output")

# -i: 輸入資料夾
# -o: 輸出資料夾
# -d 501: Dataset ID
# -c 3d_fullres: 配置
# -f 0: 使用 Fold 0 的模型 (如果你練了多個 Fold，可以寫 -f 0 1 2 3 4，它會自動做 Ensemble，準度會大增！)
# === 加分步驟：尋找最佳後處理策略 ===
print("正在分析最佳後處理策略...")
!nnUNetv2_find_best_configuration 501 -c 3d_fullres -f 0 1 2 3 4

print("開始預測測試集...")
# 預測時修改這行 (Step 6)
# -f 0 1 2 3 4 代表同時使用這 5 個 Fold 的權重投票
!nnUNetv2_predict -i {input_folder} -o {output_folder} -d 501 -c 3d_fullres -f 0 1 2 3 4

正在分析最佳後處理策略...

***All results:***
nnUNetTrainer__nnUNetPlans__3d_fullres: 0.561258911543795

*Best*: nnUNetTrainer__nnUNetPlans__3d_fullres: 0.561258911543795

***Determining postprocessing for best model/ensemble***
Results were improved by removing all but the largest foreground region. Mean dice before: 0.56126 after: 0.56138
Results were improved by removing all but the largest component for 1. Dice before: 0.92272 after: 0.92272
Results were improved by removing all but the largest component for 2. Dice before: 0.76142 after: 0.76145
Removing all but the largest component for 3 did not improve results! Dice before: 0.0 after: 0.0

***Run inference like this:***

nnUNetv2_predict -d Dataset501_Cardiac -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 1 2 3 4 -tr nnUNetTrainer -c 3d_fullres -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file C:/Users/user/AICUP\nnUNet_results\Dataset5

In [15]:
# === 步驟 7：恢復檔名並打包 ===
import zipfile
import datetime

# 設定你的比賽輸出格式要求
current_time = datetime.datetime.now().strftime("%d_%H_%M")
final_submission_dir = f"./Submission_{current_time}"
os.makedirs(final_submission_dir, exist_ok=True)

source_folder = os.path.join(results_dir, "Dataset501_Cardiac", "inference_output")

print(f"正在處理檔案並搬移至: {final_submission_dir}")

# 複製並改名
for file in glob.glob(os.path.join(source_folder, "*.nii.gz")):
    filename = os.path.basename(file)
    # 假設原本是 patient0051.nii.gz，nnU-Net 輸出也是 patient0051.nii.gz
    # 這裡依你的需求改成 patient0051_predict.nii.gz
    new_filename = filename.replace(".nii.gz", "_predict.nii.gz")
    
    shutil.copy(file, os.path.join(final_submission_dir, new_filename))

# 壓縮成 ZIP
zip_filename = f"{final_submission_dir}.zip"
print(f"正在壓縮成 {zip_filename} ...")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(final_submission_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)

print("全部完成！請上傳 ZIP 檔。")

正在處理檔案並搬移至: ./Submission_27_21_12
正在壓縮成 ./Submission_27_21_12.zip ...
全部完成！請上傳 ZIP 檔。


In [2]:
# === 步驟 A：執行 2D 配置的規劃與預處理 (低記憶體模式) ===
import os

print("[INFO] 開始進行 2D 模型的資料預處理 (限制單一程序以避免 OOM)...")

# 1. 執行預處理指令
# -d 501: Dataset ID
# -c 2d: 指定 2D 配置
# -np 1: [關鍵修改] 強制只使用 1 個 CPU 核心進行處理，大幅降低記憶體需求
# --verify_dataset_integrity: 檢查資料完整性

!nnUNetv2_plan_and_preprocess -d 501 -c 2d --verify_dataset_integrity -np 2

[INFO] 開始進行 2D 模型的資料預處理 (限制單一程序以避免 OOM)...
Fingerprint extraction...
Dataset501_Cardiac
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [0.515      0.40636719 0.40636719]. 
Current patch size: (np.int64(96), np.int64(160), np.int64(160)). 
Current median shape: [293.68932039 497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [0.53045   0.4185582 0.4185582]. 
Current patch size: (np.int64(96), np.int64(160), np.int64(1


100%|██████████| 50/50 [41:41<00:00, 50.04s/it]


In [3]:
# === 步驟 B：全自動 2D 5-Fold 訓練腳本 (防崩潰/低記憶體版) ===
import subprocess
import sys
import time
import os

# --- [關鍵修正] 設定環境變數 ---
# 將多工處理數量設為 0，強迫使用主程序 (Main Process) 處理資料
# 這能解決 Windows 下的 DLL load failed 和 Pagefile 錯誤
os.environ['nnUNet_n_proc_DA'] = '0' 
os.environ['nnUNet_def_n_proc'] = '0'

# 設定參數
folds_to_train = [0, 1, 2, 3, 4] 
dataset_id_code = 501
config_mode = "2d"

print("[INFO] 已啟動「防崩潰模式」(單線程資料讀取)...")
print("[INFO] 準備開始 2D 5-Fold 訓練 (Fold 0 ~ 4)...")

def run_fold_stable(fold, dataset_id, config):
    cmd = f"nnUNetv2_train {dataset_id} {config} {fold} --npz"
    
    print(f"\n[INFO] 開始訓練 Fold {fold}")
    
    process = subprocess.Popen(
        cmd, shell=True, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT, 
        text=True, encoding="utf-8", errors="replace"
    )
    
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        
        if line:
            # 極簡過濾邏輯
            if line.strip().startswith("Epoch:"):
                sys.stdout.write(line)
                sys.stdout.flush()
            elif "Mean" in line or "Dice" in line:
                sys.stdout.write(line)
                sys.stdout.flush()
            elif "Error" in line or "Exception" in line or "Traceback" in line:
                sys.stdout.write(line)
                sys.stdout.flush()
    
    if process.returncode != 0:
        print(f"[ERROR] Fold {fold} 訓練失敗！")
        return False
    else:
        print(f"[SUCCESS] Fold {fold} 訓練完成！")
        return True

start_time_total = time.time()

for fold in folds_to_train:
    success = run_fold_stable(fold, dataset_id_code, config_mode)
    if not success:
        print(f"[WARN] Fold {fold} 失敗，停止後續任務。")
        break

end_time_total = time.time()
hours = int((end_time_total - start_time_total) // 3600)
print(f"\n[INFO] 2D 訓練任務全部結束！總耗時: {hours} 小時")

[INFO] 已啟動「防崩潰模式」(單線程資料讀取)...
[INFO] 準備開始 2D 5-Fold 訓練 (Fold 0 ~ 4)...

[INFO] 開始訓練 Fold 0
2025-11-28 15:26:45.966471: Yayy! New best EMA pseudo Dice: 0.25940001010894775
2025-11-28 15:30:14.721807: Yayy! New best EMA pseudo Dice: 0.26190000772476196
2025-11-28 15:33:18.632852: Yayy! New best EMA pseudo Dice: 0.28060001134872437
2025-11-28 15:36:10.269117: Yayy! New best EMA pseudo Dice: 0.3027999997138977
2025-11-28 15:39:03.013253: Yayy! New best EMA pseudo Dice: 0.32330000400543213
2025-11-28 15:41:55.799750: Yayy! New best EMA pseudo Dice: 0.3424000144004822
2025-11-28 15:44:50.497889: Yayy! New best EMA pseudo Dice: 0.36000001430511475
2025-11-28 15:47:43.068893: Yayy! New best EMA pseudo Dice: 0.37529999017715454
2025-11-28 15:50:36.084206: Yayy! New best EMA pseudo Dice: 0.38830000162124634
2025-11-28 15:53:29.007337: Yayy! New best EMA pseudo Dice: 0.4018000066280365
2025-11-28 15:56:20.917130: Yayy! New best EMA pseudo Dice: 0.41519999504089355
2025-11-28 15:59:14.337406: Yayy

In [ ]:
# === 步驟 C：執行 2D 預測 (儲存機率圖以供集成) ===
import os
import shutil

# 定義路徑
input_folder = os.path.join(raw_data_dir, "Dataset501_Cardiac", "imagesTs")
# 輸出到專用的 2D 預測資料夾
output_folder_2d = os.path.join(results_dir, "Dataset501_Cardiac", "inference_output_2d")

# 清空舊資料
if os.path.exists(output_folder_2d):
    print(f"[INFO] 清空舊的 2D 預測資料夾: {output_folder_2d}")
    shutil.rmtree(output_folder_2d)
os.makedirs(output_folder_2d, exist_ok=True)

print("[INFO] 開始執行 2D 5-Fold 集成預測...")
print("[INFO] 注意：此步驟包含 --save_probabilities，會佔用較多硬碟空間。")

# 執行預測
# -c 2d: 使用 2D 模型
# -f 0 1 2 3 4: 使用 5 個 Fold
# --save_probabilities: 儲存機率檔 (Ensemble 必備)
!nnUNetv2_predict -i {input_folder} -o {output_folder_2d} -d 501 -c 2d -f 0 1 2 3 4 --save_probabilities

print("[INFO] 2D 預測完成！準備進行 3D + 2D 集成。")